## Dataset Class

The Dataset class is essentially a blueprint. When you create a
custom Dataset, you decide how data is loaded and returned.

It defines:

• __init__()-->  which tells how data should be loaded.

• __len__()-->  which returns the total number of samples.

• __getitem__(index)-->  which returns the data (and label) at the
  given index.


## DataLoader Class

The DataLoader wraps a Dataset and handles batching, shuffling,
and parallel loading for you.

### DataLoader Control Flow:

• At the start of each epoch, the DataLoader (if shuffle=True)
shuffles indices(using a sampler).

• It divides the indices into chunks of batch_size.

• for each index in the chunk, data samples are fetched from
the Dataset object

• The samples are then collected and combined into a batch
(using collate_fn)

• The batch is returned to the main training loop

In [1]:
from sklearn.datasets import make_classification
import torch

In [2]:
# Step 1: Create a synthetic classification dataset using sklearn
X, y = make_classification(
    n_samples=10,       # Number of samples
    n_features=2,       # Number of features
    n_informative=2,    # Number of informative features
    n_redundant=0,      # Number of redundant features
    n_classes=2,        # Number of classes
    random_state=42     # For reproducibility
)

In [3]:
X

array([[ 1.06833894, -0.97007347],
       [-1.14021544, -0.83879234],
       [-2.8953973 ,  1.97686236],
       [-0.72063436, -0.96059253],
       [-1.96287438, -0.99225135],
       [-0.9382051 , -0.54304815],
       [ 1.72725924, -1.18582677],
       [ 1.77736657,  1.51157598],
       [ 1.89969252,  0.83444483],
       [-0.58723065, -1.97171753]])

In [4]:
X.shape

(10, 2)

In [5]:
y

array([1, 0, 0, 0, 0, 1, 1, 1, 1, 0])

In [6]:
y.shape

(10,)

In [7]:
# Convert the data to PyTorch tensors
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.long)

In [8]:
X

tensor([[ 1.0683, -0.9701],
        [-1.1402, -0.8388],
        [-2.8954,  1.9769],
        [-0.7206, -0.9606],
        [-1.9629, -0.9923],
        [-0.9382, -0.5430],
        [ 1.7273, -1.1858],
        [ 1.7774,  1.5116],
        [ 1.8997,  0.8344],
        [-0.5872, -1.9717]])

In [9]:
y

tensor([1, 0, 0, 0, 0, 1, 1, 1, 1, 0])

In [10]:
from torch.utils.data import Dataset, DataLoader

In [15]:
class CustomDataset(Dataset):

  def __init__(self, features, labels):

    self.features = features
    self.labels = labels

  def __len__(self):

    return self.features.shape[0] # no. of rows

  def __getitem__(self, index):    # in this you can also apply transformations

    return self.features[index], self.labels[index]

In [16]:
dataset = CustomDataset(X, y)

In [17]:
len(dataset)

10

In [18]:
dataset[2]

(tensor([-2.8954,  1.9769]), tensor(0))

In [19]:
dataloader = DataLoader(dataset, batch_size=2, shuffle=False)

In [20]:
for batch_features, batch_labels in dataloader:

  print(batch_features)
  print(batch_labels)
  print("-"*50)

tensor([[ 1.0683, -0.9701],
        [-1.1402, -0.8388]])
tensor([1, 0])
--------------------------------------------------
tensor([[-2.8954,  1.9769],
        [-0.7206, -0.9606]])
tensor([0, 0])
--------------------------------------------------
tensor([[-1.9629, -0.9923],
        [-0.9382, -0.5430]])
tensor([0, 1])
--------------------------------------------------
tensor([[ 1.7273, -1.1858],
        [ 1.7774,  1.5116]])
tensor([1, 1])
--------------------------------------------------
tensor([[ 1.8997,  0.8344],
        [-0.5872, -1.9717]])
tensor([1, 0])
--------------------------------------------------


# DataLoader's Sampler and collate_fn in PyTorch

The PyTorch `DataLoader` class is a powerful utility for batch loading data during model training and evaluation. Two of its important parameters are `sampler` and `collate_fn`, which give you fine-grained control over how data is processed.

## Sampler

The `sampler` parameter controls the strategy for sampling data from your dataset.

### What it does:
- Determines the order in which data is accessed
- Controls which samples are selected in each epoch
- Returns indices that the DataLoader uses to retrieve items from the dataset

### Built-in samplers:
- `RandomSampler`: Samples elements randomly, with or without replacement
- `SequentialSampler`: Samples elements sequentially, always in the same order
- `SubsetRandomSampler`: Samples randomly from a predefined list of indices
- `WeightedRandomSampler`: Samples according to specified weights (useful for class imbalance)
- `BatchSampler`: Wraps another sampler to yield batches of indices

### Example usage:
```python
from torch.utils.data import DataLoader, RandomSampler

# Random sampling
sampler = RandomSampler(dataset)
dataloader = DataLoader(dataset, sampler=sampler, batch_size=32)

# For handling class imbalance with weighted sampling
weights = [1.0 if label == 0 else 5.0 for _, label in dataset]
sampler = WeightedRandomSampler(weights, len(weights))
dataloader = DataLoader(dataset, sampler=sampler, batch_size=32)
```

## collate_fn

The `collate_fn` parameter is a function that specifies how to combine individual samples into a batch.

### What it does:
- Takes a list of samples from your dataset
- Processes them into a batch suitable for model input
- Handles conversions, padding, and other transformations needed

### Default behavior:
- For tensors: stacks them along a new dimension
- For numpy arrays: converts to tensors, then stacks
- For numbers: converts to tensors
- For strings/custom types: keeps them in lists

### When to customize:
- Variable-length sequences (requiring padding)
- Mixed data types
- Complex data structures
- Special preprocessing for batches

### Example custom collate function:
```python
def custom_collate(batch):
    # Separate images and labels
    images = [item[0] for item in batch]
    labels = [item[1] for item in batch]
    
    # Stack images
    images = torch.stack(images, 0)
    
    # For variable-length text data, you might pad sequences
    # padded_labels = pad_sequence(labels, batch_first=True)
    
    return images, torch.tensor(labels)

dataloader = DataLoader(dataset, batch_size=32, collate_fn=custom_collate)
```

## Practical use cases

1. **Natural Language Processing**: Custom `collate_fn` for padding variable-length sequences
2. **Imbalanced datasets**: Weighted samplers to balance class representation
3. **Custom validation splits**: SubsetRandomSampler to train on specific data subsets
4. **Curriculum learning**: Custom samplers that change difficulty over epochs



#  DataLoader Parameters

The PyTorch `DataLoader` class is a versatile utility that manages the loading of data during model training and evaluation. Here's a comprehensive explanation of its most important parameters:

## Core Parameters

### `dataset` (required)
- The dataset from which to load data
- Must implement `__getitem__()` and `__len__()` methods
- Can be any map-style or iterable-style dataset

### `batch_size` (default=1)
- Number of samples in each batch
- Larger batches can speed up training but consume more memory
- Common values: 16, 32, 64, 128, 256

### `shuffle` (default=False)
- Whether to shuffle the data at the start of each epoch
- Set to `True` for training to prevent model from learning order-based patterns
- Often set to `False` for validation/testing to ensure reproducibility

### `sampler` (default=None)
- Defines the strategy to draw samples from the dataset
- If specified, `shuffle` must be `False` (they're mutually exclusive)
- Examples: `RandomSampler`, `SequentialSampler`, `WeightedRandomSampler`

### `batch_sampler` (default=None)
- Alternative to `sampler` that returns batches of indices
- If specified, `batch_size`, `shuffle`, `sampler`, and `drop_last` are ignored

### `num_workers` (default=0)
- Number of subprocesses for data loading
- `0` means data is loaded in the main process
- Increasing this can significantly speed up data loading, typically set to CPU core count

### `collate_fn` (default=None)
- Function to merge a list of samples into a mini-batch
- Default handles most tensor and numpy array conversions
- Custom functions needed for variable-length sequences or special preprocessing

### `pin_memory` (default=False)
- If `True`, tensors are copied to CUDA pinned memory before returning
- Can accelerate data transfer to GPU
- Recommended when using GPU acceleration

### `drop_last` (default=False)
- Whether to drop the last incomplete batch if dataset size isn't divisible by batch size
- Set to `True` when batch normalization is used to avoid errors with small batches
- Setting to `False` uses all data but may yield a smaller final batch

## Additional Important Parameters

### `timeout` (default=0)
- Timeout value for collecting a batch from workers
- `0` means no timeout
- Useful in distributed settings to avoid hanging

### `worker_init_fn` (default=None)
- Function to set up each worker process
- Useful for setting random seeds or worker-specific configurations

### `prefetch_factor` (default=2)
- Number of batches loaded per worker in advance
- Higher values use more memory but can improve efficiency
- Only used when `num_workers > 0`

### `persistent_workers` (default=False)
- If `True`, worker processes will not be shut down after a dataset has been consumed
- Reduces the overhead of starting and stopping workers between epochs
- Most beneficial with large datasets and many workers

### `generator` (default=None)
- Random number generator for reproducibility with shuffling

## Example with Key Parameters

```python
from torch.utils.data import DataLoader, Dataset

# Custom dataset
class MyDataset(Dataset):
    def __getitem__(self, index):
        # Return a sample
        pass
    
    def __len__(self):
        # Return dataset size
        pass

# Define DataLoader with common parameters
dataloader = DataLoader(
    dataset=MyDataset(),
    batch_size=32,
    shuffle=True,
    num_workers=4,
    pin_memory=True,
    drop_last=True,
    persistent_workers=True
)
```
